# Template for the User Tutorial


#### Developed at Volkamer Lab, Charité/FU Berlin 

Annie Pham

# Reference

### Provide some general information about your method and the source of it

Use this citation style

* Keyword describing resource: Journal (year), volume, pages (link to resource)

Example:

* ChEMBL web services: Nucleic Acids Res. (2015), 43, 612-620 (https://academic.oup.com/nar/article/43/W1/W612/2467881)

# Introduction


## What are the  chosen structures

*These points should refer to the headlines of your theory section below.*

* Names of the structures 
* Function of the structures

## Why they have been chosen

*These points should refer to the headlines of your practical section below.*

* Interesting facts (e.g. some event that happened)

# Theory

## RMSD

The RMSD is the average distance between the atoms of superposed structures in Angstrom.

## coverage

The coverage of the aligned structures.

## Details of the algorithm

### MatchMaker
    
#### Introduction:

MatchMaker superimposes protein by first creating pairwise sequence alignments, then fitting the aligned residue pairs.
The standard Needleman-Wunsch and Smith-Waterman algorithms are available for producing global and local sequence alignments and MatchMaker can identify the best-matching chains based on alignment scores.
Alignment scores can include residue similarity, secondary structure information, and gap penalties.
MatchMaker performs a spatial superposition by minimizing the RMSD.
    
#### Preparation:

Generating pairwise sequence alignments and matching, i.e., superimposing the structures according to those pairwise alignments

#### Alignment:

Spatially align the group of atoms `mobile` to `reference` by doing a RMSD fit on `select` atoms.

- A rotation matrix is computed that minimizes the RMSD between the coordinates of `mobile.select_atoms(sel1)` and `reference.select_atoms(sel2)`; before the rotation, `mobile` is translated so that its center of geometry (or center of mass)
coincides with the one of `reference`.
- All atoms in `~MDAnalysis.core.universe.Universe` that contain `mobile` are shifted and rotated.
 
#### Analysis:

RMSD before and after spatial alignment


# Preparation

## How to get the structure from the CLI

To get the structures directly from the RCSB, the syntax looks like this:

In [1]:
!structuralalignment --method=YOUR_METHOD NAME_OF_STRUCTURE_1 NAME_OF_STRUCTURE_2

usage: structuralalignment [-h] [--version] [-v]
                           [--method {theseus,mmligner,matchmaker}]
                           [--method-options METHOD_OPTIONS]
                           structures [structures ...]
structuralalignment: error: argument --method: invalid choice: 'YOUR_METHOD' (choose from 'theseus', 'mmligner', 'matchmaker')


When you want to use structures which are locally saved, do this:

In [2]:
!structuralalignment --method=YOUR_METHOD PATH_OF_STRUCTURE_1 PATH_OF_STRUCTURE_2

usage: structuralalignment [-h] [--version] [-v]
                           [--method {theseus,mmligner,matchmaker}]
                           [--method-options METHOD_OPTIONS]
                           structures [structures ...]
structuralalignment: error: argument --method: invalid choice: 'YOUR_METHOD' (choose from 'theseus', 'mmligner', 'matchmaker')


## Getting the structure in python

The method will use atomium.models as input.

If you want to get the structures from the RCSB, you can do the following:

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [73]:
import atomium

structure1 = atomium.fetch("6HG4").model
structure2 = atomium.fetch("6HG9").model

# Alignment 

Use your method and explain the steps it takes.

In [90]:
%%time
from structuralalignment.superposition.matchmaker import MatchMakerAligner, mda_align
from MDAnalysis.analysis.rms import rmsd
aligner = MatchMakerAligner(alignment_strategy="global", superposition_selection="name CA or name CB")
res = aligner.calculate([structure1, structure2])

MOB POS: [[  7.26  -74.356 -39.303]
 [  8.213 -73.765 -38.371]
 [  9.192 -74.812 -37.817]
 ...
 [ 77.479  18.555 -20.978]
 [ 79.198  17.632 -22.565]
 [ 77.004  17.186 -20.569]]
MOB POS: [[  7.26     -74.356    -39.303   ]
 [-31.060482 -41.040554 -21.117619]
 [  9.192    -74.812    -37.817   ]
 ...
 [ 77.479     18.555    -20.978   ]
 [ 79.198     17.632    -22.565   ]
 [ 77.004     17.186    -20.569   ]]
MOB POS: [[-22.207058  -49.10183   -25.805735 ]
 [-60.52754   -15.786382   -7.6203527]
 [-20.275057  -49.557823  -24.319735 ]
 ...
 [ 48.01194    43.809174   -7.480735 ]
 [ 49.73094    42.886173   -9.067735 ]
 [ 47.53694    42.440174   -7.0717344]]
CPU times: user 3.41 s, sys: 83.8 ms, total: 3.5 s
Wall time: 3.29 s


In [85]:
print(f'From RMSD = {res["metadata"]["initial_rmsd"]:.3f}A to optimized RMSD of {res["scores"]["rmsd"]:.3f}A')

From RMSD = 187.820A to optimized RMSD of 1.597A


# Analysis

### NGLview

If you have trouble with NGLview, follow this [troubleshooting guide](https://github.com/SBRG/ssbio/wiki/Troubleshooting#tips-for-nglview).

In [96]:
import nglview as nv
print("nglview version = {}".format(nv.__version__))
# your nglview version should be 1.1.7 or later

view = nv.show_mdanalysis(res["superposed"][0].atoms)
view.add_component(res["superposed"][1].atoms)

view

nglview version = 2.7.5


NGLWidget()

In [92]:
nv.MDAnalysisTrajectory??

In [88]:
view.add_component(res["superposed"][2])

In [97]:
a, b, _ = res["superposed"]

In [102]:
a.universe.trajectory[1]

IndexError: Index 1 exceeds length of trajectory (1).

In [47]:
b.write("mob.pdb")

## Report

* RMSD before and after
* coverage
* what residues are mapped